###  Sacramento Real Estate Data App

Gavin Woodgate

Updated 07/04/2024

#### Importing Dependencies

Uncomment the following cell to install the necessary dependencies with pip. It is recommended to run this notebook in a virtual environment to avoid conflicts with existing packages.

In [1]:
# ! pip install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
import os
import pyarrow.feather as feather
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Column, Spinner
from bokeh.palettes import all_palettes
from bokeh.models.widgets import CheckboxGroup, RangeSlider, DataTable, TableColumn
from bokeh.layouts import column, row

### Importing the data

The Sacramento Realestate Transactions dataset contains categorical columns including city, state, type, beds, baths, and street type. There are also numerical ints such as zip, beds, baths, sq_ft, and price as well as floats including latitude and longitude to indicate the physical location of the properties. There are no null values within this set of data for any of the columns and there are a total of 982 entries in the set. This data includes a new row called “street_type” that was derived from the “street” column by collecting the street abbreviation from the string and converting it into a categorical value.

In [3]:
df = feather.read_feather('data' + os.sep + 'cleaned_sacramento_dataset')
df['type'] = df['type'].astype('category')

In [15]:
df.head(5000)

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude,empty_lot,street_type,color
0,3526 HIGH ST,SACRAMENTO,95838,CA,2,1,836,Residential,Wed May 21 00:00:00 EDT 2008,59222,38.631913,-121.434879,False,ST,#03e8fc
1,51 OMAHA CT,SACRAMENTO,95823,CA,3,1,1167,Residential,Wed May 21 00:00:00 EDT 2008,68212,38.478902,-121.431028,False,CT,#03e8fc
2,2796 BRANCH ST,SACRAMENTO,95815,CA,2,1,796,Residential,Wed May 21 00:00:00 EDT 2008,68880,38.618305,-121.443839,False,ST,#03e8fc
3,2805 JANETTE WAY,SACRAMENTO,95815,CA,2,1,852,Residential,Wed May 21 00:00:00 EDT 2008,69307,38.616835,-121.439146,False,WAY,#03e8fc
4,6001 MCMAHON DR,SACRAMENTO,95824,CA,2,1,797,Residential,Wed May 21 00:00:00 EDT 2008,81900,38.519470,-121.435768,False,DR,#03e8fc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980,9169 GARLINGTON CT,SACRAMENTO,95829,CA,4,3,2280,Residential,Thu May 15 00:00:00 EDT 2008,232425,38.457679,-121.359620,False,CT,#03e8fc
981,6932 RUSKUT WAY,SACRAMENTO,95823,CA,3,2,1477,Residential,Thu May 15 00:00:00 EDT 2008,234000,38.499893,-121.458890,False,WAY,#03e8fc
982,7933 DAFFODIL WAY,CITRUS HEIGHTS,95610,CA,3,2,1216,Residential,Thu May 15 00:00:00 EDT 2008,235000,38.708824,-121.256803,False,WAY,#03e8fc
983,8304 RED FOX WAY,ELK GROVE,95758,CA,4,2,1685,Residential,Thu May 15 00:00:00 EDT 2008,235301,38.417000,-121.397424,False,WAY,#03e8fc


In [5]:
# Make Color Column
df['color'] = np.where(df['type'] == 'Residential', '#03e8fc', np.where(df['type'] == 'Condo', '#dd00ff', '#00ff00'))

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 984 entries, 0 to 984
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   street       984 non-null    object  
 1   city         984 non-null    category
 2   zip          984 non-null    category
 3   state        984 non-null    category
 4   beds         984 non-null    category
 5   baths        984 non-null    category
 6   sq__ft       984 non-null    int64   
 7   type         984 non-null    category
 8   sale_date    984 non-null    object  
 9   price        984 non-null    int64   
 10  latitude     984 non-null    float64 
 11  longitude    984 non-null    float64 
 12  empty_lot    984 non-null    bool    
 13  street_type  979 non-null    category
 14  color        984 non-null    object  
dtypes: bool(1), category(7), float64(2), int64(2), object(3)
memory usage: 74.7+ KB


### Introduction

This experiment extends typical EDA for the Sacramento dataset and builds a data app to explore the data further. The data app will allow users to filter on multiple criteria and makes data exploration more accessible to users and more customizable. The below cell is interactive

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
data = ColumnDataSource(df)

def make_plot(cds:ColumnDataSource, x_axis='x', y_axis='y', mark='circle', mark_size=10, color='color'):
    fig = figure(title='Sacramento Real Estate Transactions')
    fig.scatter(x=x_axis, y=y_axis, source=cds, marker=mark, size=mark_size, fill_color=color, legend_field='type')
    fig.xaxis.axis_label = 'Longitude'
    fig.yaxis.axis_label = 'Latitude'
    return fig

show(make_plot(data, x_axis='longitude', y_axis='latitude'))

### Part II: Refine ColumnDataSource Object Based on Search Criteria

In [9]:
def make_dataset(df, res_type=None, price_range=None, baths_range=None, 
                 beds_range=None, square_foot_range=None):
    
    # Copy the dataframe
    filtered_df = df.copy()

    if res_type:
        filtered_df = filtered_df[filtered_df['type'].isin(res_type)]
    
    if price_range:
        filtered_df = filtered_df[filtered_df['price'].between(price_range[0], price_range[1])]
    
    if baths_range:
        filtered_df = filtered_df[filtered_df['baths'].astype(int).between(baths_range[0], baths_range[1])]
    
    if beds_range:
        filtered_df = filtered_df[filtered_df['beds'].astype(int).between(beds_range[0], beds_range[1])]
    
    if square_foot_range:
        filtered_df = filtered_df[filtered_df['sq__ft'].between(square_foot_range[0], square_foot_range[1])] 

    return ColumnDataSource(filtered_df)


In [10]:
residential_type_criteria = ['Residential', 'Condo', 'Multi-Family']
price_criteria = [5000, 750000]
baths_criteria = [1, 6]
beds_criteria = [1, 10]
sqft_criteria = [100, 20000]

filtered_source = make_dataset(df, res_type=residential_type_criteria, 
                               price_range=price_criteria, baths_range=baths_criteria,
                               beds_range=beds_criteria, square_foot_range=sqft_criteria)
show(make_plot(filtered_source, x_axis='longitude', y_axis='latitude'))

### Part III: Add Widgets and Create an Interactive Visualization

In [11]:
controls = []
housing_selection = CheckboxGroup(labels=df['type'].unique().tolist(), active=[0])
controls.append(housing_selection)

range_slider_price = RangeSlider(start=0, end=df['price'].max(), value=(50000,75000), step=100, title="Price")
controls.append(range_slider_price)

range_slider_sq_ft = RangeSlider(start=0, end=df['sq__ft'].max(), value=(1000,2000), step=1, title="Sq_ft")
controls.append(range_slider_sq_ft)

range_slider_beds = RangeSlider(start=0, end=df['beds'].astype(int).max(), value=(1,2), step=1, title="Beds")
controls.append(range_slider_beds)

range_slider_baths = RangeSlider(start=0, end=df['baths'].astype(int).max(), value=(1,2), step=1, title="Baths")
controls.append(range_slider_baths)

In [12]:
residential_type_criteria = ['Residential', 'Condo', 'Multi-Family']
price_criteria = [5000, 750000]
baths_criteria = [1, 6]
beds_criteria = [1, 10]
sqft_criteria = [100, 20000]

source = make_dataset(df, res_type=residential_type_criteria, 
                               price_range=price_criteria, baths_range=baths_criteria,
                               beds_range=beds_criteria, square_foot_range=sqft_criteria)


def update(attr, old, new):
# Get the list of carriers for the graph
    selected_type = [housing_selection.labels[i] for i in housing_selection.active]
    price_range = [range_slider_price.value[0], range_slider_price.value[1]]
    baths_range = [range_slider_baths.value[0], range_slider_baths.value[1]]
    sq_ft_range = [range_slider_sq_ft.value[0], range_slider_sq_ft.value[1]]
    beds_range = [range_slider_beds.value[0], range_slider_beds.value[1]]
    # Make a new dataset based on the selected carriers and the
    # make_dataset function defined earlier
    new_src = make_dataset(df, selected_type, price_range, baths_range,
                           beds_range, sq_ft_range)
    # Update the source used in the quad glpyhs
    source.data.update(new_src.data)
    
    
def modify_doc(doc):
    housing_selection.on_change('active', update)
    range_slider_price.on_change('value', update)
    range_slider_baths.on_change('value', update)
    range_slider_beds.on_change('value', update)
    range_slider_sq_ft.on_change('value', update)
    doc.add_root(row(make_plot(source, x_axis='longitude', y_axis='latitude'),column(controls)))
    #If you want to add A table to the visualization
    #doc.add_root(row(figure_object,column(controls)))
show(modify_doc)

### Conclusion

Data Apps allow users to easily explore data sets and interact with filtering properties. The Data App in Part III of this notebook allows the user to filter by Residential, Condo, and Multi-Family property types as well as filter ranges of Price, Sq_ft, Beds, and Baths. Overall, data apps are accessible and customizable, which allows users to extract meaningful insights with ease and precision.